In [13]:
import pandas as pd
from pyquery import PyQuery as pq
import jinja2
import shubhlipi as sh
import os

In [14]:
# Loading the data required
data = pd.read_csv("./data/answer_key.csv", dtype=str)
data.rename(columns={"Correct Option(s)/ Answers": "CorrectAnswerID"}, inplace=True)
data.insert(0, "Index", 0)
data.insert(1, "GivenAnswer", "")
data.insert(2, "CorrectAnswer", "")
data.insert(3, "Type", "")
data["GivenAnswerID"] = ""
data["QuestionIMG"] = ""
data["Option1IMG"] = ""
data["Option2IMG"] = ""
data["Option3IMG"] = ""
data["Option4IMG"] = ""
QUESTION_URL_PREFIX = "https://cdn3.digialm.com"


In [15]:
h_data = pq(filename="./data/question_paper_html")  # html data

tables = h_data("td.rw table.menu-tbl")
# Options table, these contains the correct option number and options ids
q_tables = h_data("td.rw table.questionRowTbl")
# question table, as this contains the numerical anserws
# so will be used to get the numerical answers

OPTIONS = ["A", "B", "C", "D"]

i = 0
for table in tables:
    tbl = pq(table)
    typ = tbl("tr").eq(0)("td").eq(1).text()
    # Question ID
    q_id = tbl("tr").eq(1)("td").eq(1).text()
    # Option Number
    awns_number = 0
    # Option ID
    awns_id = "--"
    given_ans = "--"
    ques_index = data[data["QuestionID"] == q_id].index[0]
    if typ == "MCQ":
        awns_number = tbl("td").eq(15).text()
        awns_ids = [tbl("tr").eq(x + 2)("td").eq(1).text() for x in range(4)]
        data.loc[ques_index, "CorrectAnswer"] = OPTIONS[
            awns_ids.index(data.loc[ques_index, "CorrectAnswerID"])
        ]
        if awns_number != "--":
            awns_id = awns_ids[int(awns_number) - 1]
            given_ans = OPTIONS[int(awns_number) - 1]
        IMG_IDS = [x.attrib["src"] for x in q_tables.eq(i).eq(0).eq(0)("img")]
        for t_x in enumerate(
            ["QuestionIMG", "Option1IMG", "Option2IMG", "Option3IMG", "Option4IMG"]
        ):
            data.loc[ques_index, t_x[1]] = QUESTION_URL_PREFIX + IMG_IDS[t_x[0]]
    else:
        id12 = q_tables.eq(i).eq(0).text().split("\n")[-1]
        if id12 != "--":
            awns_id = id12
            given_ans = id12
    data.loc[ques_index, "GivenAnswerID"] = awns_id
    data.loc[ques_index, "Index"] = i + 1
    data.loc[ques_index, "GivenAnswer"] = given_ans
    data.loc[ques_index, "Type"] = typ if typ == "MCQ" else "NUM"
    if typ == "SA":
        data.loc[ques_index, "CorrectAnswer"] = data.loc[ques_index, "CorrectAnswerID"]
        data.loc[ques_index, "QuestionIMG"] = (
            QUESTION_URL_PREFIX + q_tables.eq(i).eq(0).eq(0)("img")[0].attrib["src"]
        )
    i += 1
data.index = data["Index"].to_numpy()
data.sort_index(inplace=True)
data.drop("Index", inplace=True, axis=1)


In [16]:
# Calculating the score

score = 0

sh.makedir("./result")

for x in ["result.csv", "result.md", "questions.md"]:
    if os.path.isfile(f"./result/{x}"):
        sh.delete_file(f"./result/{x}")

i = 1
correct_ques = []
incorrect_ques = []
unattempted_ques = []
while i <= 90:
    dt = [data.loc[i, "CorrectAnswer"], data.loc[i, "GivenAnswer"]]
    if dt[1] == "--":
        # Unanswered question are not evaluated
        unattempted_ques.append(str(i))
    elif dt[0] == dt[1]:
        # print(f"Question {i} is correct")
        score += 4
        correct_ques.append(str(i))
    else:
        # print(f"Question {i} is incorrect")
        score -= 1
        incorrect_ques.append(str(i))
    i += 1

# Saving the Results
data.to_csv("./result/result.csv")


def render_jinja_temp(fl: str, **o):
    return (
        jinja2.Environment(loader=jinja2.FileSystemLoader("../py/data/"))
        .get_template(fl)
        .render(**o)
    )


OPTIONS = {
    "correct": len(correct_ques),
    "incorrect": len(incorrect_ques),
    "unattempted": len(unattempted_ques),
    "attempted": len(correct_ques) + len(incorrect_ques),
    "score": score,
    "md_data": data.drop(
        ["Type", "QuestionIMG", "Option1IMG", "Option2IMG", "Option3IMG", "Option4IMG"],
        axis=1,
    ).to_markdown(),
    "correct_ques": correct_ques,
    "incorrect_ques": incorrect_ques,
    "unattempted_ques": unattempted_ques,
}

with open("./result/result.md", "w", encoding="utf-8") as f:
    f.write(render_jinja_temp("result_temp.md.j2", **OPTIONS))

with open("./result/questions.md", "w", encoding="utf-8") as f:
    f.write(render_jinja_temp("questions_temp.md.j2", data=data))
